In [ ]:
SYFT_VERSION = ">=0.8.2.b0,<0.9"
package_string = f'"syft{SYFT_VERSION}"'

In [ ]:
# stdlib
import os
import time

# third party
import docker
import numpy as np

# syft absolute
import syft as sy

sy.requires(SYFT_VERSION)

# syft absolute
from syft.service.worker.image_registry import SyftImageRegistry
from syft.service.worker.worker_image import SyftWorkerImage

In [ ]:
# Uncomment this to run the whole docker based custom workers
# os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "container_stack"
# os.environ["DEV_MODE"] = "True"


# Disable inmemory worker for container stack
running_as_container = os.environ.get("ORCHESTRA_DEPLOYMENT_TYPE") in (
    "container_stack",
    "k8s",
)
in_memory_workers = not running_as_container

In [ ]:
domain = sy.orchestra.launch(
    name="test-domain-1",
    dev_mode=True,
    create_producer=True,
    in_memory_workers=in_memory_workers,
    reset=True,
    port=8081,
)

In [ ]:
domain_client = domain.login(email="info@openmined.org", password="changethis")

We should see a default worker pool

In [ ]:
domain_client.worker_pools

#### Submit Dockerfile

In [ ]:
custom_dockerfile_str = f"""
FROM openmined/grid-backend:{sy.__version__}

RUN pip install pydicom

""".strip()

In [ ]:
docker_config = sy.DockerWorkerConfig(dockerfile=custom_dockerfile_str)

In [ ]:
# test image build locally
test_build_res = docker_config.test_image_build(tag="openmined/custom-worker:0.7.8")
test_build_res

In [ ]:
assert isinstance(test_build_res, sy.SyftSuccess), str(test_build_res)

In [ ]:
assert docker_config.dockerfile == custom_dockerfile_str

In [ ]:
submit_result = domain_client.api.services.worker_image.submit_dockerfile(
    docker_config=docker_config
)

In [ ]:
submit_result

In [ ]:
assert isinstance(submit_result, sy.SyftSuccess), str(submit_result)

In [ ]:
dockerfile_list = domain_client.images.get_all()
dockerfile_list

In [ ]:
assert len(domain_client.images.get_all()) == 2

In [ ]:
workerimage: SyftWorkerImage = None
for image in dockerfile_list:
    if not image.is_prebuilt and image.config.dockerfile == custom_dockerfile_str:
        workerimage = image
        break

assert isinstance(workerimage, SyftWorkerImage), str(workerimage)

In [ ]:
workerimage

#### Setup Local Registry

In [ ]:
# third party


class LocalRegistryContainer:
    def __init__(self):
        self.name = "local_registry"
        self.client = docker.from_env()

    def start(self, host_port=5678):
        existing = self.get()
        if existing:
            return existing

        result = self.client.containers.run(
            "registry:2",
            name=self.name,
            detach=True,
            ports={"5000/tcp": host_port},
            labels={"orgs.openmined.syft": "local-registry"},
        )

        return result

    def teardown(self):
        existing = self.get()
        if existing:
            existing.stop()
            existing.remove()

    def get(self):
        try:
            result = self.client.containers.get(self.name)
            if result.status == "running":
                return result
        except docker.errors.NotFound:
            return None


local_registry_container = LocalRegistryContainer()

#### Add Local Registry in Syft

In [ ]:
registry_add_result = domain_client.api.services.image_registry.add("localhost:5678")
registry_add_result

In [ ]:
assert isinstance(registry_add_result, sy.SyftSuccess), str(registry_add_result)

In [ ]:
images = domain_client.api.services.image_registry.get_all()
assert len(images) == 1
images

In [ ]:
local_registry = images[0]
local_registry

In [ ]:
assert isinstance(local_registry, SyftImageRegistry), str(local_registry)

#### Build Image

In [ ]:
docker_tag = "openmined/custom-worker:0.7.8"

registry_uid = local_registry.id if running_as_container else local_registry.id

docker_build_result = domain_client.api.services.worker_image.build(
    image_uid=workerimage.id,
    tag=docker_tag,
    registry_uid=registry_uid,
)
docker_build_result

In [ ]:
assert isinstance(docker_build_result, sy.SyftSuccess), str(docker_build_result)

In [ ]:
image_list = domain_client.images.get_all()
image_list

In [ ]:
for image in image_list:
    if image.id == workerimage.id:
        workerimage = (
            image  # we can also index with string using the repo_with_tag format
        )

if running_as_container:
    image_list[workerimage.built_image_tag]
    assert image_list[workerimage.built_image_tag] == workerimage

workerimage

In [ ]:
def get_image_hash(tag) -> str:
    client = docker.from_env()
    try:
        image = client.images.get(tag)
        return image.id
    except docker.errors.ImageNotFound:
        return None

In [ ]:
if running_as_container:
    assert workerimage.image_hash == get_image_hash(
        workerimage.built_image_tag
    ), "Worker Image image_hash does not match with built image hash"

#### Push Image to Local Registry

In [ ]:
push_result = None
if running_as_container:
    # stdlib
    from time import sleep

    local_registry_container.start()
    sleep(5)

    push_result = domain_client.api.services.worker_image.push(workerimage.id)
    assert isinstance(push_result, sy.SyftSuccess), str(push_result)

In [ ]:
push_result

In [ ]:
if running_as_container:
    # third party
    import requests

    base_url = f"http://{workerimage.image_identifier.registry_host}"
    expected_tag = workerimage.image_identifier.tag

    repos = requests.get(f"{base_url}/v2/_catalog").json()["repositories"]
    tags = requests.get(f"{base_url}/v2/openmined/custom-worker/tags/list").json()
    tags = tags["tags"]

    assert (
        "openmined/custom-worker" in repos
    ), f"'openmined/custom-worker' not uploaded to local registry | {repos}"
    assert (
        expected_tag in tags
    ), f"'openmined/custom-worker' with tag {expected_tag} not available | {tags}"

#### Delete locally built image to force pull from local registry

This should make the subsequent `worker_pool.launch` pull from registry at 'localhost:5678`

In [ ]:
# stdlib
from time import sleep


def remove_local_image(tag):
    client = docker.from_env()
    try:
        client.images.remove(tag)
    except docker.errors.ImageNotFound:
        pass


if running_as_container:
    remove_local_image(workerimage.built_image_tag)
    sleep(5)

#### Create Worker Pool From Image

In [ ]:
worker_pool_name = "my_first_worker_pool"
worker_pool_res = domain_client.api.services.worker_pool.launch(
    name=worker_pool_name,
    image_uid=workerimage.id,
    num_workers=3,
)

In [ ]:
assert len(worker_pool_res) == 3

In [ ]:
for status in worker_pool_res:
    assert status.error is None
    if running_as_container:
        assert status.worker.image.image_hash == get_image_hash(
            workerimage.built_image_tag
        ), "Worker Pool Image image_hash does not match with built image hash"

In [ ]:
worker_pool_list = domain_client.worker_pools
worker_pool_list

In [ ]:
assert len(domain_client.worker_pools.get_all()) == 2
worker_pool = None
for pool in worker_pool_list:
    if pool.name == worker_pool_name:
        worker_pool = pool
        break
assert worker_pool is not None
assert len(worker_pool.workers) == 3

In [ ]:
# We can filter pools based on the image id upon which the pools were built
domain_client.api.services.worker_pool.filter_by_image_id(image_uid=workerimage.id)

In [ ]:
# Delete the second worker
second_worker = worker_pool.workers[1]

In [ ]:
second_worker

#### Get Worker Logs

In [ ]:
raw_worker_logs = domain_client.api.services.worker.logs(
    uid=second_worker.id,
    raw=True,
)
raw_worker_logs

In [ ]:
assert isinstance(raw_worker_logs, bytes)

In [ ]:
worker_logs = domain_client.api.services.worker.logs(
    uid=second_worker.id,
)
worker_logs

In [ ]:
assert isinstance(worker_logs, str)

#### Delete Worker from Pool

In [ ]:
worker_delete_res = domain_client.api.services.worker.delete(
    uid=second_worker.id,
)

In [ ]:
worker_delete_res

In [ ]:
assert isinstance(worker_delete_res, sy.SyftSuccess), str(worker_delete_res)

In [ ]:
# Refetch the worker pool
# Ensure that the deleted worker's id is not present
for pool in domain_client.api.services.worker_pool.get_all():
    if pool.name == worker_pool_name:
        worker_pool = pool
assert len(worker_pool.workers) == 2
for worker in worker_pool.workers:
    assert second_worker.id != worker.id

In [ ]:
worker_pool

### Syft function

In [ ]:
data = np.array([1, 2, 3])
data_action_obj = sy.ActionObject.from_obj(data)

data_pointer = domain_client.api.services.action.set(data_action_obj)
data_pointer

In [ ]:
@sy.syft_function(
    input_policy=sy.ExactMatch(x=data_pointer),
    output_policy=sy.SingleExecutionExactOutput(),
    worker_pool_name=worker_pool_name,
)
def custom_worker_func(x):
    return {"y": x + 1}

In [ ]:
custom_worker_func

In [ ]:
assert custom_worker_func.worker_pool_name == worker_pool.name

In [ ]:
request = domain_client.code.request_code_execution(custom_worker_func)
request

In [ ]:
domain_client.requests[-1].approve(approve_nested=True)

In [ ]:
job = domain_client.code.custom_worker_func(x=data_pointer, blocking=False)
job

In [ ]:
worker_pool = domain_client.worker_pools[worker_pool_name]
worker_pool

In [ ]:
job.wait()

In [ ]:
assert job.status.value == "completed"

In [ ]:
job = domain_client.jobs[-1]
job

In [ ]:
job.job_worker_id

In [ ]:
# Disabling it due to Race Condition Error
# assert job.job_worker_id is not None

In [ ]:
# Sleeping so that consumer state is updated
time.sleep(5)

In [ ]:
# Once the work is done by the worker, its state is returned to idle again.
consuming_worker_is_now_idle = False
for worker in domain_client.worker_pools[worker_pool_name].workers:
    if worker.id == job.job_worker_id:
        consuming_worker_is_now_idle = worker.consumer_state.value.lower() == "idle"

assert consuming_worker_is_now_idle is True

In [ ]:
# Validate the result received from the syft function
result = job.wait().get()
result_matches = result["y"] == data + 1
assert result_matches.all()

#### Worker Image

In [ ]:
# delete the remaining workers
for worker in worker_pool.workers:
    res = domain_client.api.services.worker.delete(
        uid=worker.id,
    )
    assert isinstance(res, sy.SyftSuccess), str(res)

# Adding some sleep to allow containers to be fully removed,
# before removing the image
time.sleep(10)

In [ ]:
delete_res = domain_client.api.services.worker_image.remove(workerimage.id)

In [ ]:
delete_res

In [ ]:
# Since the containers are delete, we should be able to delete the image
assert isinstance(delete_res, sy.SyftSuccess), str(delete_res)
delete_res

In [ ]:
if running_as_container:
    local_registry_container.teardown()

#### Worker Pool and Image Creation Request/Approval

In [ ]:
custom_dockerfile_str_2 = f"""
FROM openmined/grid-backend:{sy.__version__}

RUN pip install opendp
""".strip()

docker_config_2 = sy.DockerWorkerConfig(dockerfile=custom_dockerfile_str_2)

In [ ]:
submit_result = domain_client.api.services.worker_image.submit_dockerfile(
    docker_config=docker_config_2
)
submit_result

In [ ]:
domain_client.images

In [ ]:
# get the image that's not built
workerimage_2 = None
for im in domain_client.images:
    if im.config == docker_config_2:
        workerimage_2 = im

##### Build image first then create pool

In [ ]:
docker_tag_2 = "openmined/custom-worker-opendp:latest"

docker_build_result = domain_client.api.services.worker_image.build(
    image_uid=workerimage_2.id,
    tag=docker_tag_2,
)
docker_build_result

In [ ]:
pool_create_request = domain_client.api.services.worker_pool.pool_creation_request(
    pool_name="first-opendp-pool", num_workers=3, image_uid=workerimage_2.id
)
pool_create_request

In [ ]:
assert len(pool_create_request.changes) == 1

In [ ]:
# get the pending request and approve it
req_result = pool_create_request.approve()
req_result

In [ ]:
assert isinstance(req_result, sy.SyftSuccess), str(req_result)

In [ ]:
assert domain_client.worker_pools["first-opendp-pool"]
assert len(domain_client.worker_pools["first-opendp-pool"].worker_list) == 3

In [ ]:
assert len(domain_client.worker_pools.get_all()) == 3

##### Request to build the image and create the pool at the same time

In [ ]:
custom_dockerfile_str_3 = f"""
FROM openmined/grid-backend:{sy.__version__}

RUN pip install recordlinkage
""".strip()

docker_config_3 = sy.DockerWorkerConfig(dockerfile=custom_dockerfile_str_3)

docker_tag_3 = "openmined/custom-worker-recordlinkage:latest"

In [ ]:
# syft absolute
from syft.custom_worker.builder import CustomWorkerBuilder
from syft.service.response import SyftError
from syft.service.response import SyftSuccess


def test_image_build(config: str, tag: str, **kwargs):
    builder = CustomWorkerBuilder()
    try:
        result = builder.build_image(
            config=config, tag=tag, pull=True, rm=True, forcerm=True, **kwargs
        )
        return SyftSuccess(message=result.logs)
    except Exception as e:
        return SyftError(message=f"Failed to build image !! Error: {str(e)}.")


test_build_res = test_image_build(config=docker_config_3, tag=docker_tag_3)
assert isinstance(test_build_res, sy.SyftSuccess), str(test_build_res)

In [ ]:
pool_image_create_request = (
    domain_client.api.services.worker_pool.create_image_and_pool_request(
        pool_name="recordlinkage-pool",
        num_workers=2,
        tag=docker_tag_3,
        config=docker_config_3,
        reason="I want to do some more cool data science with PySyft and OpenDP",
    )
)
pool_image_create_request

In [ ]:
assert len(pool_image_create_request.changes) == 2
assert pool_image_create_request.changes[0].config == docker_config_3
assert pool_image_create_request.changes[1].num_workers == 2
assert pool_image_create_request.changes[1].pool_name == "recordlinkage-pool"

In [ ]:
# get the pending request and approve it
req_result = pool_image_create_request.approve()
req_result

In [ ]:
assert isinstance(req_result, sy.SyftSuccess), str(req_result)

In [ ]:
# Get updated request object and status
for req in domain_client.requests:
    if req.id == pool_image_create_request.id:
        pool_image_create_request = req

assert pool_image_create_request.status.value == 2

In [ ]:
domain_client.images

In [ ]:
image_exists = False
for im in domain_client.images.get_all():
    if im.image_identifier and im.image_identifier.repo_with_tag == docker_tag_3:
        image_exists = True
assert image_exists

In [ ]:
assert domain_client.worker_pools["recordlinkage-pool"]
assert len(domain_client.worker_pools["recordlinkage-pool"].worker_list) == 2

#### Clean up workers

In [ ]:
# delete the remaining workers
for worker_pool in domain_client.worker_pools:
    for worker in worker_pool.workers:
        res = domain_client.api.services.worker.delete(uid=worker.id, force=True)
        print(res)
        assert isinstance(res, sy.SyftSuccess), str(res)

# Adding some sleep to allow containers to be fully removed,
# before removing the image
time.sleep(10)

In [ ]:
for worker_pool in domain_client.worker_pools:
    assert len(worker_pool.worker_list) == 0

In [ ]:
domain.land()